In [1]:
%cd yolov5
import torch
from yolov5 import utils
display = utils.notebook_init() 

YOLOv5 🚀 v6.1-325-g3e85863 Python-3.9.12 torch-1.12.0 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)


Setup complete ✅ (20 CPUs, 7.6 GB RAM, 45.9/251.0 GB disk)


In [15]:
print('Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Using torch 1.12.0 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3070 Laptop GPU', major=8, minor=6, total_memory=8191MB, multi_processor_count=40)


In [2]:
import xml.etree.ElementTree as ET
import glob
import os
import json

In [20]:
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]

In [17]:
%cd /home/ray/9444/project/data
classes = []
input_dir = "/home/ray/9444/project/data/annotations"
output_dir = "./labels/"
image_dir = "/home/ray/9444/project/data/images"
os.mkdir(output_dir)

/home/ray/9444/project/data


In [21]:
files = glob.glob(os.path.join(input_dir, '*.xml'))
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    if not os.path.exists(os.path.join(image_dir, f"{filename}.png")):
        print(f"{filename} image does not exist!")
        continue
    
    result = []
    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)
    
    for obj in root.findall('object'):
        label = obj.find("name").text
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")
        
    if result:
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))

In [22]:
# generate the classes file as reference
with open('classes.txt', 'w', encoding='utf8') as f:
    f.write(json.dumps(classes))

Split the Train/Test Dataset

In [3]:
import os
import numpy as np

import shutil
%cd /home/ray/9444/project/
root_dir = "/home/ray/9444/project/dataset/"
labels_dir = "/home/ray/9444/project/data/labels"
images_dir = "/home/ray/9444/project/data/images"

/home/ray/9444/project


In [36]:
%cd /home/ray/9444/project/
os.mkdir(root_dir)

for i in ["train", "val","test"]:
    os.mkdir(root_dir+i)

/home/ray/9444/project


In [39]:
images_list = list(sorted(os.listdir(images_dir)))
labels_list = list(sorted(os.listdir(labels_dir)))

In [40]:
len(images_list)

853

In [50]:
np.random.seed(23)
np.random.shuffle(images_list)
np.random.seed(23)
np.random.shuffle(labels_list)

In [51]:
for i in range(len(images_list)):
    if i < len(images_list) * 0.7:
        shutil.copy(os.path.join(images_dir, images_list[i]), os.path.join(root_dir, "train", images_list[i]))
        shutil.copy(os.path.join(labels_dir, labels_list[i]), os.path.join(root_dir, "train", labels_list[i]))
    elif i < len(images_list) * 0.85:
        shutil.copy(os.path.join(images_dir, images_list[i]), os.path.join(root_dir, "test", images_list[i]))
        shutil.copy(os.path.join(labels_dir, labels_list[i]), os.path.join(root_dir, "test", labels_list[i]))
    else:
        shutil.copy(os.path.join(images_dir, images_list[i]), os.path.join(root_dir, "val", images_list[i]))
        shutil.copy(os.path.join(labels_dir, labels_list[i]), os.path.join(root_dir, "val", labels_list[i]))

In [53]:
len(os.listdir(root_dir+"test"))

256

In [55]:
text_to_write = "train: /home/ray/9444/project/dataset/train\n"
text_to_write += "val: /home/ray/9444/project/dataset/val\n"
text_to_write += "test: /home/ray/9444/project/dataset/test\n\n"
text_to_write += "# number of classes\nnc: 3\n\n"
text_to_write += "# class names\nnames: [\"with_mask\", \"mask_weared_incorrect\", \"without_mask\"]\n"

In [58]:
with open("/home/ray/9444/project/dataset/data_face_mask.yaml",'w') as f:
    print(f.write(text_to_write))

230


In [60]:
import torch
print('Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Using torch 1.12.0 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3070 Laptop GPU', major=8, minor=6, total_memory=8191MB, multi_processor_count=40)


In [61]:
data_dir = '/home/ray/9444/project/dataset/'

In [62]:
%cd /home/ray/9444/project/yolov5
project = 'yolov5_mask_detection'
name = 'default_yolov5m_pretrained_coco'

/home/ray/9444/project/yolov5


In [74]:
%%time
!python train.py --img 640 --batch 8 --epochs 200 --data {data_dir}data_face_mask.yaml --weights yolov5m.pt --cache --project {project} --name {name}

train: weights=yolov5m.pt, cfg=, data=/home/ray/9444/project/dataset/data_face_mask.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5_mask_detection, name=default_yolov5m_pretrained_coco, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-325-g3e85863 Python-3.9.12 torch-1.12.0 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0,

In [75]:
best_weight = '/home/ray/9444/project/yolov5/yolov5_mask_detection/default_yolov5m_pretrained_coco/weights/best.pt'
%cd /home/ray/9444/project/yolov5
!python3 val.py --weights {best_weight} --data /home/ray/9444/project/dataset/data_face_mask.yaml --img 640 --iou 0.65 --task test

/home/ray/9444/project/yolov5
val: data=/home/ray/9444/project/dataset/data_face_mask.yaml, weights=['/home/ray/9444/project/yolov5/yolov5_mask_detection/default_yolov5m_pretrained_coco/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.65, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-325-g3e85863 Python-3.9.12 torch-1.12.0 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)

Fusing layers... 
Model summary: 290 layers, 20861016 parameters, 0 gradients, 47.9 GFLOPs
test: Scanning '/home/ray/9444/project/dataset/test.cache' images and labels... 
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        128        562      0.811      0.728      0.762      0.495
           with_mask        128        428      0.944      0.923       0.

In [73]:
!python detect.py --weights {best_weight} --img 640 --conf 0.50 --source /home/ray/9444/project/dataset/test/

detect: weights=['/home/ray/9444/project/yolov5/yolov5_mask_detection/default_yolov5m_pretrained_coco4/weights/best.pt'], source=/home/ray/9444/project/dataset/test/, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-325-g3e85863 Python-3.9.12 torch-1.12.0 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)

Fusing layers... 
Model summary: 290 layers, 20861016 parameters, 0 gradients, 47.9 GFLOPs
image 1/128 /home/ray/9444/project/dataset/test/maksssksksss102.png: 384x640 1 with_mask, Done. (0.021s)
image 2/128 /home/ray/9444/project/dataset/test/maksssksksss107.png: 640x512 1 with_mask, Done. (0.016s)
image 3/128 /home/ray/9444/project/dataset/t

In [76]:
%cd /home/ray/9444/project/yolov5
project = 'yolov5_mask_detection'
name = 'default_yolov5s_pretrained_coco'

/home/ray/9444/project/yolov5


In [80]:
%%time
!python train.py --img 480 --batch 8 --epochs 200 --data {data_dir}data_face_mask.yaml --weights yolov5s.pt --cache --project {project} --name {name}

train: weights=yolov5s.pt, cfg=, data=/home/ray/9444/project/dataset/data_face_mask.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=8, imgsz=480, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5_mask_detection, name=default_yolov5s_pretrained_coco, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-325-g3e85863 Python-3.9.12 torch-1.12.0 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0,

In [81]:
%%time
!python train.py --img 640 --batch 8 --epochs 200 --data {data_dir}data_face_mask.yaml --weights yolov5s.pt --cache --project {project} --name {name}

train: weights=yolov5s.pt, cfg=, data=/home/ray/9444/project/dataset/data_face_mask.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5_mask_detection, name=default_yolov5s_pretrained_coco, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-325-g3e85863 Python-3.9.12 torch-1.12.0 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0,

In [5]:
best_weight = '/home/ray/9444/project/yolov5/yolov5_mask_detection/default_yolov5s_pretrained_coco/weights/best.pt'
%cd /home/ray/9444/project/yolov5
!python3 val.py --weights {best_weight} --data /home/ray/9444/project/dataset/data_face_mask.yaml --img 640 --iou 0.65 --task test

/home/ray/9444/project/yolov5
val: data=/home/ray/9444/project/dataset/data_face_mask.yaml, weights=['/home/ray/9444/project/yolov5/yolov5_mask_detection/default_yolov5s_pretrained_coco/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.65, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-325-g3e85863 Python-3.9.12 torch-1.12.0 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)

Fusing layers... 
Model summary: 213 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning '/home/ray/9444/project/dataset/test.cache' images and labels... 
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        128        562      0.748      0.702      0.749      0.488
           with_mask        128        428      0.892      0.928      0.95

In [4]:
best_weight = '/home/ray/9444/project/yolov5/yolov5_mask_detection/default_yolov5s_pretrained_coco2/weights/best.pt'
%cd /home/ray/9444/project/yolov5
!python3 val.py --weights {best_weight} --data /home/ray/9444/project/dataset/data_face_mask.yaml --img 640 --iou 0.65 --task test

/home/ray/9444/project/yolov5
val: data=/home/ray/9444/project/dataset/data_face_mask.yaml, weights=['/home/ray/9444/project/yolov5/yolov5_mask_detection/default_yolov5s_pretrained_coco2/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.65, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-325-g3e85863 Python-3.9.12 torch-1.12.0 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)

Fusing layers... 
Model summary: 213 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning '/home/ray/9444/project/dataset/test.cache' images and labels... 
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        128        562      0.807      0.752      0.793      0.505
           with_mask        128        428      0.903      0.911      0.9

In [6]:
best_weight = '/home/ray/9444/project/yolov5/yolov5_mask_detection/default_yolov5s_pretrained_coco2/weights/best.pt'
!python detect.py --weights {best_weight} --img 640 --conf 0.50 --source /home/ray/9444/project/dataset/test/

detect: weights=['/home/ray/9444/project/yolov5/yolov5_mask_detection/default_yolov5s_pretrained_coco2/weights/best.pt'], source=/home/ray/9444/project/dataset/test/, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-325-g3e85863 Python-3.9.12 torch-1.12.0 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)

Fusing layers... 
Model summary: 213 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
image 1/128 /home/ray/9444/project/dataset/test/maksssksksss102.png: 384x640 2 with_masks, Done. (0.010s)
image 2/128 /home/ray/9444/project/dataset/test/maksssksksss107.png: 640x512 1 with_mask, Done. (0.011s)
image 3/128 /home/ray/9444/project/dataset/t